<h1>Read the linearly seperable classification dataset files and prepare the data for training, testing, and validation</h1>

In [41]:
import pandas as pd
import numpy as np
import os
import cv2
import collections
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder

# reading all images in a directory
path1 = r"D:\Sujeet_PhD\Course_Work\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Classification\Image_Group32\train\movie_theater_indoor"
path2 = r"D:\Sujeet_PhD\Course_Work\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Classification\Image_Group32\train\rock_arch"
path3 = r"D:\Sujeet_PhD\Course_Work\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Classification\Image_Group32\train\valley"
path4 = r"D:\Sujeet_PhD\Course_Work\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Classification\Image_Group32\test\movie_theater_indoor"
path5 = r"D:\Sujeet_PhD\Course_Work\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Classification\Image_Group32\test\rock_arch"
path6 = r"D:\Sujeet_PhD\Course_Work\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Classification\Image_Group32\test\valley"

dirs1 = os.listdir(path1)
dirs2 = os.listdir(path2)
dirs3 = os.listdir(path3)
dirs4 = os.listdir(path4)
dirs5 = os.listdir(path5)
dirs6 = os.listdir(path6)

BoVW_train_class1, BoVW_train_class2, BoVW_train_class3 = [], [], []
BoVW_test_class1, BoVW_test_class2, BoVW_test_class3 = [], [], []

def BoVW_feature(dirs, path, BoVW_class):
    windowsize_r = 32
    windowsize_c = 32
    for file in dirs:
        img = cv2.imread(os.path.join(path, file), 1)
        [h, w, c] = img.shape
        patches = []
        store_patch = []
        patch_vec = []
        for r in range(0, h - windowsize_r, windowsize_r):
            for c in range(0, w - windowsize_c, windowsize_c):
                window = img[r:r+windowsize_r, c:c+windowsize_c]
                patches.append(window)
        for r in range(0, h - windowsize_r, windowsize_r):
            window = img[r:r+windowsize_r, w-windowsize_c:w]
            patches.append(window)
        for c in range(0, w - windowsize_c, windowsize_c):
            window = img[h-windowsize_r:h, c:c+windowsize_c]
            patches.append(window)
        window = img[h-windowsize_r:h, w-windowsize_c:w]
        patches.append(window)
            
        for j in range(0, len(patches)):
            hist_b = cv2.calcHist([patches[j]], [0], None, [8], [0, 256])
            hist_g = cv2.calcHist([patches[j]], [1], None, [8], [0, 256])
            hist_r = cv2.calcHist([patches[j]], [2], None, [8], [0, 256])
            hist = np.concatenate((hist_b, hist_g, hist_r))
            store_patch.append(hist.T[0])
        
        kmeans = KMeans(n_clusters = 32, random_state = 0).fit(np.array(store_patch))
        patch_vec = kmeans.predict(np.array(store_patch))
        counter = list(collections.Counter(patch_vec).values())
        BoVW_class.append(counter)
                
### Call BoVW_feature function to prepare the BoVW dataset from images of all classes
BoVW_feature(dirs1, path1, BoVW_train_class1)
BoVW_feature(dirs2, path2, BoVW_train_class2)
BoVW_feature(dirs3, path3, BoVW_train_class3)
BoVW_feature(dirs4, path4, BoVW_test_class1)
BoVW_feature(dirs5, path5, BoVW_test_class2)
BoVW_feature(dirs6, path6, BoVW_test_class3)

### Create data set by combining all three class data
def all_class_data(train1, train2, train3, validate1, validate2, validate3, test1, test2, test3):
    trainX = np.concatenate((train1, train2, train3), axis=0)
    trainY = np.array([0 for i in range(len(train1))] + [1 for i in range(len(train2))] + [2 for i in range(len(train3))])
    validateX = np.concatenate((validate1, validate2, validate3), axis=0)
    validateY = np.array([0 for i in range(len(validate1))] + [1 for i in range(len(validate2))] + [2 for i in range(len(validate3))])
    testX = np.concatenate((test1, test2, test3), axis=0)
    testY = np.array([0 for i in range(len(test1))] + [1 for i in range(len(test2))] + [2 for i in range(len(test3))])
    return trainX, trainY, validateX, validateY, testX, testY

### Devide the Class data into training, validation, and testing data
train1, validate1 = np.split(np.array(BoVW_train_class1), [int(0.8*len(BoVW_train_class1))])
test1 = np.array(BoVW_test_class1)
### Devide the Class2 data into training, validation, and testing data
train2, validate2 = np.split(np.array(BoVW_train_class2), [int(0.8*len(BoVW_train_class2))])
test2 = np.array(BoVW_test_class2)
### Devide the Class3 data into training, validation, and testing data
train3, validate3 = np.split(np.array(BoVW_train_class3), [int(0.8*len(BoVW_train_class3))])
test3 = np.array(BoVW_test_class3)
trainX, trainY, validateX, validateY, testX, testY = all_class_data(train1, train2, train3, validate1, validate2, validate3, test1, test2, test3)

### Convert label data to one hot encoder
### 0 -> (1, 0, 0), 1 -> (0, 1, 0), 2 -> (0, 0, 1)
enc = OneHotEncoder()
y_OHE_train = enc.fit_transform(np.expand_dims(trainY,1)).toarray()
y_OHE_val = enc.fit_transform(np.expand_dims(validateY,1)).toarray()
y_OHE_test = enc.fit_transform(np.expand_dims(testY,1)).toarray()
print("Done")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you us

Done


<h1>Implementation of multilayer feed forward neural network (MLFFNN)</h1> 

In [69]:
import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

### Activation Functions Definitions
class Sigmoid():
    def __call__(self, x, b=1):
        return 1.0/(1.0 + np.exp(-(b*x)))
    def gradient(self, x, b=1):
        return self.__call__(x, b) * (1 - self.__call__(x, b))

class Linear():
    def __call__(self, x, b=1):
        return b*x
    def gradient(self, x, b=1):
        return b

### Multilayer feed forward neural network class
class MLFFNN():
    def __init__(self, n_hidden, n_epoch=1000, learning_rate=0.01, threshold=0.001):
        self.n_hidden = n_hidden
        self.n_epoch = n_epoch
        self.learning_rate = learning_rate
        self.threshold = threshold
        self.hidden_activation = Sigmoid()
        self.output_activation = Sigmoid()

    ### Initialize the weights of neural network
    def initialize_weights(self, X, y):
        n_samples, n_features = X.shape
        _, n_outputs = y.shape
        
        ### For all hidden layers
        pre_num_of_neuron = n_features
        self.weights = {}
        self.w0 = {}
        for i in range(len(self.n_hidden)):
            limit   = 1 / math.sqrt(pre_num_of_neuron/2)
            self.weights[i]  = np.random.uniform(-limit, limit, (pre_num_of_neuron, self.n_hidden[i]))
            self.w0[i] = np.random.uniform(-limit, limit, (1, self.n_hidden[i]))
            pre_num_of_neuron = self.n_hidden[i]
        
        # For output layer
        limit   = 1 / math.sqrt(pre_num_of_neuron/2)
        self.V  = np.random.uniform(-limit, limit, (self.n_hidden[-1], n_outputs))
        self.v0 = np.random.uniform(-limit, limit, (1, n_outputs))

    def train(self, X, y, epoch=True):
        self.initialize_weights(X, y)
        self.errors = []
        ### This conditional block of code is for fixed number of epoch
        if epoch == True:
            ### Run it for n_epoch times
            for i in range(self.n_epoch):
                ### For hidden layer
                inputs = X
#                 ### Randomize the input data
#                 rand_index = np.random.permutation(len(inputs))
#                 inputs = inputs[rand_index]
#                 y = y[rand_index]
                
                self.hidden_input = {}
                self.hidden_output = {}
                ### Forward Calculation ###
                self.forward_calculation(inputs)
                ### Backpropagation Calculation ###
                self.backpropagation_calculation(inputs, y)

                ### Store average instantaneous errors for each epoch
                self.errors.append(np.sum(self.SquareLoss(y, self.y_pred))/y.shape[0])
        ### This conditional block of code is for fixed threshold of average error
        else:
            error = 10000000
            noOfNoChangeError = 0
            ### Run it until error converges to the threshhold
            while error > self.threshold:
                ### For hidden layer
                inputs = X
#                 ### Randomize the input data
#                 rand_index = np.random.permutation(len(inputs))
#                 inputs = inputs[rand_index]
#                 y = y[rand_index]
                
                self.hidden_input = {}
                self.hidden_output = {}
                ### Forward Calculation ###
                self.forward_calculation(inputs)
                ### Backpropagation Calculation ###
                self.backpropagation_calculation(inputs, y)
                
                ### Store average instantaneous errors for each epoch
                self.errors.append(np.sum(self.SquareLoss(y, self.y_pred))/y.shape[0])
                ### If there is no change in error
                if noOfNoChangeError >=20:
                    break
                else:
                    if len(self.errors) >= 4:
                        if error == self.errors[-1] and error == self.errors[-2]:
                            noOfNoChangeError += 1
                error = self.errors[-1]
                
    ### Forward Calculation ###
    def forward_calculation(self, inputs):
        ### For hidden layer
        for i in range(len(self.n_hidden)):
            ### Input to neuron
            self.hidden_input[i] = inputs.dot(self.weights[i]) + self.w0[i]
            ### Output of neuron
            self.hidden_output[i] = self.hidden_activation(self.hidden_input[i])
            inputs = self.hidden_output[i]
        ### For output layer
        self.output_layer_input = inputs.dot(self.V) + self.v0
        self.y_pred = self.output_activation(self.output_layer_input)
        return self.y_pred
    
    ### Backpropagation Calculation ###
    def backpropagation_calculation(self, inputs, y):
        ### First for output layer
        ### Gradient w.r.t input of output layer
        grad_wrt_out_l_input = self.loss(y, self.y_pred) * self.output_activation.gradient(self.output_layer_input)
        grad_v = self.hidden_output[len(self.n_hidden)-1].T.dot(grad_wrt_out_l_input)
        grad_v0 = np.sum(grad_wrt_out_l_input, axis=0, keepdims=True)
        ### For hidden layer
        ### Gradient w.r.t input of hidden layer
        next_grad_wrt_hidden_l_input = grad_wrt_out_l_input
        next_weight = self.V
        prev_input = inputs
        grad_w = {}
        grad_w0 = {}
        ### Calculation for multiple hidden layer starting from last to first hidden layer
        for i in reversed(range(len(self.n_hidden))):
            grad_wrt_hidden_l_input = next_grad_wrt_hidden_l_input.dot(next_weight.T) * self.hidden_activation.gradient(self.hidden_input[i])
            ### If hidden layer not connected to input layer
            if i != 0:
                grad_w[i] = self.hidden_output[i-1].T.dot(grad_wrt_hidden_l_input)
            ### when hidden layer connected to input layer
            else:
                grad_w[i] = inputs.T.dot(grad_wrt_hidden_l_input)
            grad_w0[i] = np.sum(grad_wrt_hidden_l_input, axis=0, keepdims=True)
            next_grad_wrt_hidden_l_input = grad_wrt_hidden_l_input
            next_weight = self.weights[i]

        ### Calculaton for weights update ###
        ### Weights update of output layer
        self.V  -= self.learning_rate * grad_v
        self.v0 -= self.learning_rate * grad_v0
        ### Weights update of hidden layers
        for i in range(len(self.n_hidden)):
            self.weights[i]  -= self.learning_rate * grad_w[i]
            self.w0[i] -= self.learning_rate * grad_w0[i]
    
    ### Prediction Function
    def predict(self, X):
        ### Call Forward Calculation ###
        y_pred = self.forward_calculation(X)
        return y_pred
    
    ### Instantaneous error and loss function
    def SquareLoss(self, y, y_pred):
        return 0.5 * np.power((y - y_pred), 2)
    def loss(self, y, y_pred):
        return -(y - y_pred)

### Calculate the accuracy
def accuracy_score(y, y_pred):
    accuracy = np.sum(y == y_pred, axis=0) / len(y)
    return accuracy*100

### To calculate the confusion matrix and classification accuracy
def confusion_matrix(actual, predicted):
    cm = np.zeros((3, 3))
    for i, j in zip(actual, predicted):
        cm[i][j] += 1
    ### For classification accuracy
    accuracy = np.sum(actual == predicted) * 100.0 / float(len(actual))
    return cm, accuracy

### Plot of Epoch vs Mean Square Error
def epochVsError_plot(model):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    error = model.errors
    nepoch = [i+1 for i in range(len(error))]
    plt.scatter(nepoch, error, marker='o', s=5, facecolors='b', edgecolors='b')
    plt.xlabel('Number of Epoch')
    plt.ylabel('Average Error')
#     plt.title('Average error vs number of epoch for training of Image Classification data using MLFFNN')
    plt.savefig("AvgErrorVsEpoch_MLFFNN_Image.png", dpi=600, bbox_inches="tight")
    plt.clf()

### 3D scatter plot for output of each nueron of hidden layer and output layer plot
def output_nueron_plot(X_train, y_train, z, label='hidden', layer='', nn=''):
    # Creating figure
    fig = plt.figure(figsize = (16, 9))
    ax = plt.axes(projection ="3d")
    # Add x, y gridlines 
    ax.grid(b = True, color ='grey', linestyle ='-.', linewidth = 0.3, alpha = 0.2) 
    # Creating color map
    color = ['red', 'green', 'blue']
    color_list = [color[i] for i in y_train]
    # Creating plot
    sctt = ax.scatter3D(X_train[:,0], X_train[:,1], z, color = color_list)
    ax.set_xlabel('X-axis', fontweight ='bold') 
    ax.set_ylabel('Y-axis', fontweight ='bold') 
    ax.set_zlabel('Z-axis (Neuron Output) ', fontweight ='bold')
    # save plot
    plt.savefig("Output_of_{}_layer_{}_Nueron_{}_Image.png".format(label, layer, nn), dpi=600, bbox_inches="tight")
    plt.clf()

def main():
    ### Call the MLFFNN calss 
    mlffnn = MLFFNN(n_hidden=[32,32], n_epoch=10000, learning_rate=0.001, threshold=0.001)
    ### Train the MLFFNN
    mlffnn.train(trainX, y_OHE_train, epoch=True)
    
    ### Prediction for validation data
    y_pred_train = np.argmax(mlffnn.predict(trainX), axis=1)
    y_train = np.argmax(y_OHE_train, axis=1)
    ### Prediction for validation data
    y_pred_val = np.argmax(mlffnn.predict(validateX), axis=1)
    y_val = np.argmax(y_OHE_val, axis=1)
    y1 = mlffnn.predict(validateX)
    for i in range(len(y_OHE_val)):
        print(y_OHE_val[i], y1[i])
    ### Prediction for test data
    y_pred_test = np.argmax(mlffnn.predict(testX), axis=1)
    y_test = np.argmax(y_OHE_test, axis=1)
    
    ### Calculate the accuracy and confusion matrix for validate and test data
    CM, Accuracy = confusion_matrix(y_train, y_pred_train)
    print("Confusion Matrix of training data: {}".format(CM))
    print("Classification Accuracy of training data: {}".format(Accuracy))
    CM, Accuracy = confusion_matrix(y_val, y_pred_val)
    print("Confusion Matrix of validation data: {}".format(CM))
    print("Classification Accuracy of Validation data: {}".format(Accuracy))
    CM, Accuracy = confusion_matrix(y_test, y_pred_test)
    print("Confusion Matrix of test data: {}".format(CM))
    print("Classification Accuracy of test data: {}".format(Accuracy))

    ### Epoch vs error plot
    epochVsError_plot(mlffnn)
    
if __name__ == "__main__":
    main()

[1. 0. 0.] [0.01204232 0.1219622  0.8793246 ]
[1. 0. 0.] [0.00177521 0.76258958 0.57435438]
[1. 0. 0.] [0.53628919 0.08837069 0.06590816]
[1. 0. 0.] [0.00275335 0.13820374 0.96589472]
[1. 0. 0.] [0.00328482 0.08865551 0.97398237]
[1. 0. 0.] [0.49172007 0.01524636 0.39051158]
[1. 0. 0.] [0.99911139 0.00407186 0.00112187]
[1. 0. 0.] [0.00286931 0.15785921 0.95641476]
[1. 0. 0.] [0.12885456 0.0070249  0.9230571 ]
[1. 0. 0.] [0.09517114 0.07054218 0.59313538]
[0. 1. 0.] [0.10582344 0.09138958 0.45625566]
[0. 1. 0.] [0.1092762  0.00560365 0.95203018]
[0. 1. 0.] [6.84010322e-04 9.98379099e-01 1.02150425e-02]
[0. 1. 0.] [0.12084402 0.68674333 0.01759625]
[0. 1. 0.] [0.0661278  0.01794275 0.88484413]
[0. 1. 0.] [0.72475653 0.03549654 0.08897108]
[0. 1. 0.] [0.00449028 0.06889598 0.97339825]
[0. 1. 0.] [0.00209922 0.09619329 0.98426839]
[0. 1. 0.] [0.58225978 0.01721745 0.24751539]
[0. 1. 0.] [0.0281348  0.98473555 0.00181299]
[0. 0. 1.] [0.0022039  0.99747546 0.00471701]
[0. 0. 1.] [0.02971879

<Figure size 432x288 with 0 Axes>